In [51]:
import pandas as pd
import json

# チームIDをチーム名に変換する辞書作成
team_df = pd.read_csv('/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/team.csv')
team_dic = dict(zip(team_df['team_id'], team_df['team_name']))
team_id_list = [v for v in team_dic.keys()]


# ○: フル出場, ▲: 途中出場, ▽: 途中退場, ◆: 途中出場途中退場, ×: 出場停止, ※: 他大会の影響で出場停止, B: ベンチ入り
# 各々の回数をカウント
tmp = ['rat_full_play', 'rat_out_play', 'rat_in_play', 'rat_inout_play', 'rat_bench_play', 'rat_susp_play']
colnames = ['rat_full_play', 'rat_out_play', 'rat_in_play', 'rat_inout_play', 'rat_bench_play', 'rat_susp_play']
characters = ['○', '▲', '▽', '◆', 'B', '×']

#2015の出場時間を生成
#記号を出場時間に変換する関数を定義
#前半２６分で交代した場合は”△２６”のようにデータとして保存されているので出場時間だけ取り出す
def convert_sign(self):
    import re
    
    if "○" in self:
        return int(90)
    
    elif "▽" in self:        
        num = re.sub(r"\D", "", self)
        return int(num)

    elif "▲" in self:        
        num = re.sub(r"\D", "", self)
        num2 = 90 - int(num)
        return int(num2)    
    
    else:
        return int(0)
pretime_2018 = pd.DataFrame({"name":[], "pretime":[]})
pretime_2017 = pd.DataFrame({"name":[], "pretime":[]})    
pretime_2016 = pd.DataFrame({"name":[], "pretime":[]})    
pretime_2015 = pd.DataFrame({"name":[], "pretime":[]})    

df_dic = {2018:pretime_2018, 2017:pretime_2017, 2016:pretime_2016, 2015:pretime_2015}
for year in [2018,2017,2016,2015]:
    # 2015シーズンのjsonをcsvに変換
    #year = 2015
    team_df_year = team_df[team_df.year == year]
    df = pd.DataFrame({"name":[],
                     "pretime":[]})
    for div, team_id in zip(team_df_year["div"], team_df_year["team_id"]):
        # 2stageかどうか
        is_2stage = year < 2017 and div == "J1"

        # 後で１試合あたりの指標に変換するために試合数の情報を記憶
        if div == 'J1':
            num_match = 34
        else:
            num_match = 42

        if is_2stage:
            with open(f"/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/event/event_play_{year}_{div}_{team_id}_s1.json", encoding="utf-8_sig") as f:
                first_dic = json.load(f)
            with open(f"/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/event/event_play_{year}_{div}_{team_id}_s2.json", encoding="utf-8_sig") as f:
                second_dic = json.load(f)

            first_df = pd.DataFrame.from_dict(first_dic, orient='index')
            second_df = pd.DataFrame.from_dict(second_dic, columns=list(range(17, 34)), orient='index')
            #(first_df)
            full_df = pd.concat([first_df, second_df], axis=1, join='outer')

        else:
            with open(f"/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/event/event_play_{year}_{div}_{team_id}.json", encoding="utf-8_sig") as f:
                dic = json.load(f)

            full_df = pd.DataFrame.from_dict(dic, orient='index')

        #データを四分割するための数値
        num_to_div_list = [(num_match//4)*n for n in range(1,5)]
        first = num_to_div_list[0]
        second = num_to_div_list[1]
        third = num_to_div_list[2]
        forth = num_match

        full_df = full_df.fillna("0")    
        full_df_con = full_df.applymap(convert_sign)
        full_df_con["pretime"] = full_df_con.sum(axis=1)
        full_df_con["1st_pretime"] = full_df_con.iloc[:, :first].sum(axis=1)
        full_df_con["2st_pretime"] = full_df_con.iloc[:, first:second].sum(axis=1)
        full_df_con["3st_pretime"] = full_df_con.iloc[:, second:third].sum(axis=1)
        full_df_con["4st_pretime"] = full_df_con.iloc[:, third:num_match].sum(axis=1)

        new_df = pd.DataFrame()
        new_df["name"] = full_df_con.index
        new_df["pretime"] = full_df_con["pretime"].values
        new_df["1st_pretime"] = full_df_con["1st_pretime"].values
        new_df["2st_pretime"] = full_df_con["2st_pretime"].values
        new_df["3st_pretime"] = full_df_con["3st_pretime"].values
        new_df["4st_pretime"] = full_df_con["4st_pretime"].values
        new_df['team_id'] = team_id
        new_df['team'] = new_df['team_id'].map(int).map(team_dic)
        df = pd.concat([df, new_df])

    #pd.set_option('display.max_rows',1500)
    df["name-team"] = df["name"] + "-" + df["team"]
    df.drop(["name"], axis=1, inplace=True)
    #pretime_2015.drop(["team_id"], axis=1, inplace=True)
    df.drop(["team"], axis=1, inplace=True)
    
    df["time_rate"] = df["pretime"].copy()
    df["1st_time_rate"] = df["1st_pretime"].copy()
    df["2st_time_rate"] = df["2st_pretime"].copy()
    df["3st_time_rate"] = df["3st_pretime"].copy()
    df["4st_time_rate"] = df["4st_pretime"].copy()
    for i in team_id_list:
        s = df.loc[df["team_id"]==i, "pretime"].sum()
        df.loc[df["team_id"]==i, "time_rate"] /= (s/100)
        s = df.loc[df["team_id"]==i, "1st_pretime"].sum()
        df.loc[df["team_id"]==i, "1st_time_rate"] /= (s/100)
        s = df.loc[df["team_id"]==i, "2st_pretime"].sum()
        df.loc[df["team_id"]==i, "2st_time_rate"] /= (s/100)
        s = df.loc[df["team_id"]==i, "3st_pretime"].sum()
        df.loc[df["team_id"]==i, "3st_time_rate"] /= (s/100)
        s = df.loc[df["team_id"]==i, "4st_pretime"].sum()
        df.loc[df["team_id"]==i, "4st_time_rate"] /= (s/100)
    df_dic[year] = df

In [52]:
pretime_2018 = df_dic[2018]
pretime_2017 = df_dic[2017]
pretime_2016 = df_dic[2016]
pretime_2015 = df_dic[2015]

In [53]:
pretime_2018

,pretime,1st_pretime,2st_pretime,3st_pretime,4st_pretime,team_id,name-team,time_rate,1st_time_rate,2st_time_rate,3st_time_rate,4st_time_rate
0,2430.0,720.0,540.0,360.0,810.0,1.0,クォン スンテ-鹿島,7.222041,9.090909,6.829392,4.545455,8.181818
1,602.0,161.0,326.0,89.0,26.0,1.0,内田 篤人-鹿島,1.789164,2.032828,4.122929,1.123737,0.262626
2,1378.0,540.0,437.0,41.0,360.0,1.0,昌子 源-鹿島,4.095462,6.818182,5.526748,0.517677,3.636364
3,1553.0,183.0,278.0,630.0,462.0,1.0,レオ シルバ-鹿島,4.615568,2.310606,3.515872,7.954545,4.666667
4,1260.0,720.0,540.0,0.0,0.0,1.0,植田 直通-鹿島,3.744762,9.090909,6.829392,0.000000,0.000000
5,990.0,0.0,0.0,450.0,540.0,1.0,チョン スンヒョン-鹿島,2.942313,0.000000,0.000000,5.681818,5.454545
6,1908.0,285.0,412.0,496.0,715.0,1.0,永木 亮太-鹿島,5.670639,3.598485,5.210573,6.262626,7.222222
7,226.0,144.0,82.0,0.0,0.0,1.0,ペドロ ジュニオール-鹿島,0.671679,1.818182,1.037056,0.000000,0.000000
8,2052.0,613.0,464.0,556.0,419.0,1.0,土居 聖真-鹿島,6.098612,7.739899,5.868218,7.020202,4.232323
9,2376.0,582.0,602.0,633.0,559.0,1.0,鈴木 優磨-鹿島,7.061551,7.348485,7.613507,7.992424,5.646465


In [22]:
full_df_con

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,pretime
本間 幸司,0,0,0,90,90,90,90,90,90,90,...,90,90,90,90,90,90,90,90,90,3105
田向 泰輝,0,56,0,0,90,0,0,0,0,90,...,90,90,90,90,90,90,90,90,77,2559
田中 雄大,90,90,90,90,90,90,89,90,90,90,...,90,90,90,90,90,90,90,90,90,3762
山﨑 貴雅,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
金 聖基,0,90,0,90,90,90,90,90,90,90,...,0,0,0,0,0,0,0,0,0,1450
石神 幸征,71,0,90,90,0,90,90,90,89,11,...,90,0,1,90,75,90,0,0,0,1470
鈴木 雄斗,65,69,56,0,0,31,0,0,90,73,...,79,77,90,81,90,90,69,90,90,1789
岩尾 憲,90,90,90,90,90,90,90,90,90,90,...,90,90,90,90,90,90,90,0,90,3690
小谷野 顕治,0,0,25,0,0,0,0,8,0,59,...,11,0,0,0,0,0,0,0,0,373
船谷 圭祐,0,0,0,0,0,0,0,0,9,79,...,67,90,72,78,89,76,34,90,80,2375


In [33]:
full_df_con.iloc[:,1:10].sum(axis=1)

本間　幸司      630
田向　泰輝      236
田中　雄大      809
山﨑　貴雅        1
金　聖基       720
石神　幸征      550
鈴木　雄斗      319
岩尾　憲       810
小谷野　顕治      92
船谷　圭祐       88
三島　康平      536
麦倉　捺木        0
宮市　剛       245
角口　大征        0
パク　カンイル     79
新里　亮       810
白井　永地        0
山村　佑樹      105
今瀬　淳也      707
笠原　昂史      180
内田　航平      593
岡田　明久        0
細川　淳矢       90
池ヶ谷　颯斗       0
ユン　ヨンスン      0
宋　株熏         0
福留　健吾        0
石川　大徳        0
吉田　眞紀人     656
馬場　賢治      654
鈴木　武蔵        0
dtype: int64

In [40]:
pretime_2015["time_rate"] = pretime_2015["pretime"].copy()
pretime_2015["1st_time_rate"] = pretime_2015["1st_pretime"].copy()
pretime_2015["2st_time_rate"] = pretime_2015["2st_pretime"].copy()
pretime_2015["3st_time_rate"] = pretime_2015["3st_pretime"].copy()
pretime_2015["4st_time_rate"] = pretime_2015["4st_pretime"].copy()
for i in team_id_list:
    s = pretime_2015.loc[pretime_2015["team_id"]==i, "pretime"].sum()
    pretime_2015.loc[pretime_2015["team_id"]==i, "time_rate"] /= (s/100)
    s = pretime_2015.loc[pretime_2015["team_id"]==i, "1st_pretime"].sum()
    pretime_2015.loc[pretime_2015["team_id"]==i, "1st_time_rate"] /= (s/100)
    s = pretime_2015.loc[pretime_2015["team_id"]==i, "2st_pretime"].sum()
    pretime_2015.loc[pretime_2015["team_id"]==i, "2st_time_rate"] /= (s/100)
    s = pretime_2015.loc[pretime_2015["team_id"]==i, "3st_pretime"].sum()
    pretime_2015.loc[pretime_2015["team_id"]==i, "3st_time_rate"] /= (s/100)
    s = pretime_2015.loc[pretime_2015["team_id"]==i, "4st_pretime"].sum()
    pretime_2015.loc[pretime_2015["team_id"]==i, "4st_time_rate"] /= (s/100)    
pretime_2015.head(22)

,pretime,1st_pretime,2st_pretime,3st_pretime,4st_pretime,team_id,name-team,time_rate,1st_time_rate,2st_time_rate,3st_time_rate,4st_time_rate
0,3060.0,720.0,720.0,720.0,900.0,10.0,林 卓人-広島,9.090909,9.090909,9.090909,9.090909,9.090909
1,2933.0,720.0,720.0,706.0,787.0,10.0,水本 裕貴-広島,8.713607,9.090909,9.090909,8.914141,7.949495
2,3060.0,720.0,720.0,720.0,900.0,10.0,千葉 和彦-広島,9.090909,9.090909,9.090909,9.090909,9.090909
3,2951.0,720.0,630.0,701.0,900.0,10.0,青山 敏弘-広島,8.767083,9.090909,7.954545,8.851010,9.090909
4,438.0,371.0,0.0,55.0,12.0,10.0,森﨑 浩司-広島,1.301248,4.684343,0.000000,0.694444,0.121212
5,2940.0,716.0,617.0,707.0,900.0,10.0,森﨑 和幸-広島,8.734403,9.040404,7.790404,8.926768,9.090909
6,2529.0,476.0,563.0,594.0,896.0,10.0,ドウグラス-広島,7.513369,6.010101,7.108586,7.500000,9.050505
7,2191.0,565.0,477.0,521.0,628.0,10.0,佐藤 寿人-広島,6.509210,7.133838,6.022727,6.578283,6.343434
8,0.0,0.0,0.0,0.0,0.0,10.0,増田 卓也-広島,0.000000,0.000000,0.000000,0.000000,0.000000
9,2482.0,700.0,531.0,463.0,788.0,10.0,ミキッチ-広島,7.373737,8.838384,6.704545,5.845960,7.959596


## 以下、コードの確認作業。

In [16]:
team_id_list

[1,
 10,
 12,
 13,
 14,
 18,
 20,
 21,
 22,
 3,
 31,
 33,
 46,
 5,
 54,
 7,
 8,
 9,
 11,
 2,
 23,
 24,
 27,
 275,
 277,
 28,
 29,
 330,
 338,
 34,
 36,
 37,
 39,
 4,
 40,
 42,
 45,
 47,
 78,
 94,
 38,
 48,
 35,
 43]

In [14]:
s

48550.0

In [157]:
full_df

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
本間 幸司,B,B,B,○,○,○,○,○,○,○,...,○,○,○,○,○,○,○,○,○,○
田向 泰輝,,▽56,B,B,○,B,B,B,B,○,...,○,○,○,○,○,○,○,○,○,▽77
田中 雄大,○,○,○,○,○,○,▽89,○,○,○,...,○,○,○,○,○,○,○,○,○,○
山﨑 貴雅,,B,,,B,B,▲1,B,,,...,,,,,,,,,,
金 聖基,B,○,B,○,○,○,○,○,○,○,...,B,B,B,B,,,,,,
石神 幸征,◆71,×,○,○,B,○,○,○,▽89,▲11,...,○,○,B,▲1,○,▽75,○,,,
鈴木 雄斗,▽65,▽69,▽56,B,B,▲31,B,B,○,▽73,...,○,▽79,▽77,○,▽81,○,○,▽69,○,○
岩尾 憲,○,○,○,○,○,○,○,○,○,○,...,○,○,○,○,○,○,○,○,×,○
小谷野 顕治,B,B,▲25,B,B,B,B,▲8,B,▽59,...,,▲11,,,B,B,B,B,B,B
船谷 圭祐,,,,,,,,,▲9,▽79,...,▲37,▽67,○,▽72,▽78,▽89,▽76,▲34,○,▽80


In [143]:
new_df = pd.DataFrame()
new_df["name"] = full_df_con.index
new_df["time_played"] = full_df_con["time_played"].values
new_df

,name,time_played
0,本間 幸司,3105
1,田向 泰輝,2559
2,田中 雄大,3762
3,山﨑 貴雅,89
4,金 聖基,1970
5,石神 幸征,1707
6,鈴木 雄斗,2063
7,岩尾 憲,3690
8,小谷野 顕治,815
9,船谷 圭祐,2563


In [151]:
a = pd.DataFrame({"name":[],
                 "time_played":[]})
a = pd.concat([a,new_df])
a

,name,time_played
0,本間 幸司,3105.0
1,田向 泰輝,2559.0
2,田中 雄大,3762.0
3,山﨑 貴雅,89.0
4,金 聖基,1970.0
5,石神 幸征,1707.0
6,鈴木 雄斗,2063.0
7,岩尾 憲,3690.0
8,小谷野 顕治,815.0
9,船谷 圭祐,2563.0


In [142]:
a = pd.DataFrame()
new_df = pd.DataFrame()
new_df["name"] = full_df_con.index
new_df["time_played"] = full_df_con["time_played"].values
a.concat(new_df)
#a = pd.DataFrame(a)
a

AttributeError: 'DataFrame' object has no attribute 'concat'

In [136]:
new_df = pd.DataFrame()
new_df["name"] = 0
new_df["time_played"] = 0
new_df.append(full_df_con.index)
new_df

TypeError: cannot concatenate object of type '<class 'pandas.core.indexes.base.Index'>'; only Series and DataFrame objs are valid

In [130]:
name = []
name.append(full_df_con.index)
name
time_played = []
time_played.append(full_df_con["time_played"].values)
time_played
a = pd.DataFrame({"name":name,
                 "time_played":time_played})
a

,name,time_played
0,"Index(['本間 幸司', '田向 泰輝', '田中 雄大', '山﨑 貴雅', '金 ...","[3105, 2559, 3762, 89, 1970, 1707, 2063, 3690,..."


In [92]:
type(df)

pandas.core.frame.DataFrame

In [89]:
a = pd.DataFrame({"name":[],
                 "time_played":[]})
a

,name,time_played


In [77]:
data

[林　卓人      3060
 水本　裕貴     2933
 千葉　和彦     3060
 青山　敏弘     2951
 森﨑　浩司      664
 森﨑　和幸     2940
 ドウグラス     2807
 佐藤　寿人     2191
 増田　卓也        0
 ミキッチ      2566
 工藤　浩平       84
 柏　好文      2664
 佐々木　翔     1838
 皆川　佑介      403
 野津田　岳人    1376
 茶島　雄介      222
 清水　航平     1320
 丸谷　拓也      579
 浅野　拓磨     1943
 柴﨑　晃誠     2257
 塩谷　司      2381
 宮原　和也      270
 森島　司         0
 山岸　智       332
 dtype: int64,
 藤田　優人        777
 近藤　直也        578
 鈴木　大輔       2987
 増嶋　竜也        135
 山中　亮輔        852
 大谷　秀和       2722
 茨田　陽生       2259
 工藤　壮人       2953
 大津　祐樹        991
 レアンドロ       1204
 エドゥアルド      2492
 狩野　健太        209
 武富　孝介       2171
 稲田　康志          0
 秋野　央樹       1217
 中川　寛斗        382
 中谷　進之介       266
 菅野　孝憲       2641
 輪湖　直樹       2471
 小林　祐介       1011
 太田　徹郎       1583
 キム　チャンス     2772
 栗澤　僚一       1536
 中山　雄太         68
 クリスティアーノ    3026
 大島　康樹         70
 滝本　晴彦          0
 熊川　翔           0
 浮田　健誠          0
 湯澤　聖人          0
 桐畑　和繁        391
 エデルソン        540
 今井　智基        264
 dtype:

In [19]:
df

,rat_full_play,rat_out_play,rat_in_play,rat_inout_play,rat_bench_play,rat_susp_play,team,name
0,3060.0,0.0,0.0,0.0,0.0,0.0,広島,林 卓人
1,2790.0,0.0,2.0,0.0,0.0,0.0,広島,水本 裕貴
2,3060.0,0.0,0.0,0.0,0.0,0.0,広島,千葉 和彦
3,2880.0,0.0,1.0,0.0,0.0,1.0,広島,青山 敏弘
4,270.0,4.0,2.0,0.0,3.0,0.0,広島,森﨑 浩司
...,...,...,...,...,...,...,...,...
26,0.0,0.0,0.0,0.0,0.0,0.0,水戸,福留 健吾
27,0.0,5.0,9.0,0.0,4.0,0.0,水戸,石川 大徳
28,810.0,8.0,12.0,0.0,2.0,0.0,水戸,吉田 眞紀人
29,1800.0,2.0,18.0,0.0,1.0,1.0,水戸,馬場 賢治


In [3]:
full_df

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
本間 幸司,B,B,B,○,○,○,○,○,○,○,...,○,○,○,○,○,○,○,○,○,○
田向 泰輝,,▽56,B,B,○,B,B,B,B,○,...,○,○,○,○,○,○,○,○,○,▽77
田中 雄大,○,○,○,○,○,○,▽89,○,○,○,...,○,○,○,○,○,○,○,○,○,○
山﨑 貴雅,,B,,,B,B,▲1,B,,,...,,,,,,,,,,
金 聖基,B,○,B,○,○,○,○,○,○,○,...,B,B,B,B,,,,,,
石神 幸征,◆71,×,○,○,B,○,○,○,▽89,▲11,...,○,○,B,▲1,○,▽75,○,,,
鈴木 雄斗,▽65,▽69,▽56,B,B,▲31,B,B,○,▽73,...,○,▽79,▽77,○,▽81,○,○,▽69,○,○
岩尾 憲,○,○,○,○,○,○,○,○,○,○,...,○,○,○,○,○,○,○,○,×,○
小谷野 顕治,B,B,▲25,B,B,B,B,▲8,B,▽59,...,,▲11,,,B,B,B,B,B,B
船谷 圭祐,,,,,,,,,▲9,▽79,...,▲37,▽67,○,▽72,▽78,▽89,▽76,▲34,○,▽80


In [31]:
"▽" in full_df[1][1]

True

In [72]:
characters = ['○', '▲', '▽', '◆', 'B', '×'] 
def convert_sign(self):
    import re
    
    if "○" in self:
        return int(90)
    
    elif "▽" in self:        
        num = re.sub(r"\D", "", self)
        return int(num)

    elif "▲" in self:        
        num = re.sub(r"\D", "", self)
        num2 = 90 - int(num)
        return int(num2)    
    
    else:
        return int(0)
    

full_df_con = full_df.applymap(convert_sign)
full_df_con["time_played"] = full_df_con.sum(axis=1)
full_df_con
#new_full_df_con
#print(type(full_df_con[1][1]))


,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,time_played
本間 幸司,0,0,0,90,90,90,90,90,90,90,...,90,90,90,90,90,90,90,90,90,3105
田向 泰輝,0,56,0,0,90,0,0,0,0,90,...,90,90,90,90,90,90,90,90,77,2559
田中 雄大,90,90,90,90,90,90,89,90,90,90,...,90,90,90,90,90,90,90,90,90,3762
山﨑 貴雅,0,0,0,0,0,0,89,0,0,0,...,0,0,0,0,0,0,0,0,0,89
金 聖基,0,90,0,90,90,90,90,90,90,90,...,0,0,0,0,0,0,0,0,0,1970
石神 幸征,0,0,90,90,0,90,90,90,89,79,...,90,0,89,90,75,90,0,0,0,1707
鈴木 雄斗,65,69,56,0,0,59,0,0,90,73,...,79,77,90,81,90,90,69,90,90,2063
岩尾 憲,90,90,90,90,90,90,90,90,90,90,...,90,90,90,90,90,90,90,0,90,3690
小谷野 顕治,0,0,65,0,0,0,0,82,0,59,...,79,0,0,0,0,0,0,0,0,815
船谷 圭祐,0,0,0,0,0,0,0,0,81,79,...,67,90,72,78,89,76,56,90,80,2563


In [67]:
data =[]
full_df_con_sum = full_df_con.sum(axis=1)
data.append(full_df_con_sum)
data
data_pd = pd.DataFrame(data).T
data_pd

,0
本間 幸司,6210
田向 泰輝,5118
田中 雄大,7524
山﨑 貴雅,178
金 聖基,3940
石神 幸征,3414
鈴木 雄斗,4126
岩尾 憲,7380
小谷野 顕治,1630
船谷 圭祐,5126


In [55]:
print(full_df_con.dtypes)

0     int64
1     int64
2     int64
3     int64
4     int64
5     int64
6     int64
7     int64
8     int64
9     int64
10    int64
11    int64
12    int64
13    int64
14    int64
15    int64
16    int64
17    int64
18    int64
19    int64
20    int64
21    int64
22    int64
23    int64
24    int64
25    int64
26    int64
27    int64
28    int64
29    int64
30    int64
31    int64
32    int64
33    int64
34    int64
35    int64
36    int64
37    int64
38    int64
39    int64
40    int64
41    int64
dtype: object


In [57]:
new_df = pd.DataFrame()
new_df["name"] = full_df_con.index
time_played = full_df_con.sum(axis=1)
time_played.name = "time_played"
new_df.append(time_played)
new_df

,name
0,本間 幸司
1,田向 泰輝
2,田中 雄大
3,山﨑 貴雅
4,金 聖基
5,石神 幸征
6,鈴木 雄斗
7,岩尾 憲
8,小谷野 顕治
9,船谷 圭祐


In [62]:
new_df = pd.DataFrame()
new_df["name"] = full_df_con.index
new_df["time_played"] = full_df_con.sum(axis=1).astype("int64")
new_df.dtypes

name            object
time_played    float64
dtype: object

In [37]:
def convert_sign(self):
    import re
    
    if "▽" in self:        
        num = re.sub(r"\D", "", self)
        return num
        
    
print(convert_sign(full_df[1][1]))

56


In [35]:
import re

s = "▽56"
result = re.sub(r"\D", "", s)
print(result)

56
